# MiniGPT in PyTorch
Today, we'll be creating a decoder only transformer model to generate Shakespearean style text. The transformer model was first proposed in the paper [Attention Is All You Need](https://arxiv.org/pdf/1706.03762) which was released by Google and originally intended for machine translation. The original transformer model included an encoder and decoder as exemplified by T5 and BART. The decoder only transformer model was popularlized by OpenAI's GPT series such as GPT-2.

In [ ]:
!wget https://github.com/raymond-zeng/MiniGPT/blob/main/input.txt

--2025-04-01 22:39:12--  https://github.com/raymond-zeng/MiniGPT/blob/main/input.txt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘input.txt’

input.txt               [  <=>               ]   1.41M  5.59MB/s    in 0.3s    

2025-04-01 22:39:14 (5.59 MB/s) - ‘input.txt’ saved [1482575]



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm

In [ ]:
#hyperparameters
batch_size = 64
block_size = 256
max_iters = 3000
eval_interval = 500
lr = 5e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
embed_size = 384
num_heads = 6
num_layers = 6

In [ ]:
#very very simple tokenization

torch.manual_seed(0)

with open('input.txt', 'r') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}
encode = lambda s : [char_to_idx[ch] for ch in s]
decode = lambda x : ''.join([idx_to_char[i] for i in x])

In [ ]:
#split itnto train and validation and batches

data = torch.tensor(encode(text), dtype=torch.long, device=device)
n = int(0.9 * data.size(0))
train_data, val_data = data[:n], data[n:]

def getBatch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

Transformers biggest innovation was the use of Multi-Head Attention. Multi-Head Attention essentially calculates which of the previous tokens are most important. This is done by keeping key, value pairs and queries. Attention is calculated by the formula: $$\text{softmax}(\frac{QK^T}{\sqrt{d_k}})V$$
where $QK^T$ is the simularity between two tokens. To ensure that the attention mechanism doesn't cheat and look at future tokens, we use a lower triangular matrix as a mask, so that we only calculate attention on previously seen tokens.

In [ ]:
class Head(nn.Module):

    def __init__(self, embed_size, head_size):
        super().__init__()
        self.key = nn.Linear(embed_size, head_size, bias = False)
        self.query = nn.Linear(embed_size, head_size, bias = False)
        self.value = nn.Linear(embed_size, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        B, T, C = x.size()
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        weights = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        weights = weights.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        weights = F.softmax(weights, dim=-1)
        weights = self.dropout(weights)
        return weights @ v


In [ ]:
class MultiHeadAttention(nn.Module):

    def __init__(self, embed_size, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(embed_size, head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, embed_size)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        out =  self.proj(torch.cat([head(x) for head in self.heads], dim=-1))
        return self.dropout(out)

Here we see the concept of residual connections which help deal with optimization issues such as gradient vanishing and any difficulties from training deep neural networks. They work by bypassing intermediate layers and adding the original input back to the output of these layers, forming a shortcut connection. This improves gradient flow and helps deep networks learn identity mappings more easily.

In [ ]:
class Block(nn.Module):

    def __init__(self, embed_size, num_heads):
        super().__init__()
        head_size = embed_size // num_heads
        self.attention = MultiHeadAttention(embed_size, num_heads, head_size)
        self.net = nn.Sequential(
            nn.Linear(embed_size, 4 * embed_size),
            nn.GELU(),
            nn.Linear(4 * embed_size, embed_size),
            nn.Dropout(0.2)
        )
        self.norm1 = nn.RMSNorm(embed_size)
        self.norm2 = nn.RMSNorm(embed_size)

    def forward(self, x):
        x = x + self.attention(self.norm1(x))
        x = x + self.net(self.norm2(x))
        return x

In [ ]:
class GPT(nn.Module):

    def __init__(self, vocab_size, embed_size, num_heads, num_layers):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, embed_size)
        self.pos_emb = nn.Embedding(block_size, embed_size)
        self.blocks = nn.Sequential(*[Block(embed_size, num_heads) for _ in range(num_layers)])
        self.norm = nn.RMSNorm(embed_size)
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_emb(idx)
        pos_emb = self.pos_emb(torch.arange(T, device=device))
        x = pos_emb + tok_emb
        x = self.blocks(x)
        x = self.norm(x)
        logits = self.fc(x)

        if targets is None:
          loss = None
        else:
          B, T, C = logits.shape
          logits = logits.view(B * T, C)
          targets = targets.view(B * T)
          loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_tokens):
        for _ in range(max_tokens):
          idx_cond = idx[:, -block_size:]
          logits, loss = self(idx_cond)
          logits = logits[:, -1, :]
          probs = F.softmax(logits, dim=-1)
          idx_next = torch.multinomial(probs, num_samples=1)
          idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [ ]:
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = getBatch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

model = GPT(vocab_size, embed_size, num_heads, num_layers).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

for i in tqdm.tqdm(range(max_iters)):
    X, Y = getBatch('train')
    logits, loss = model(X, Y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i % eval_interval == 0 or i == max_iters - 1:
        losses = estimate_loss(model)
        print(f"train loss: {losses['train']:.4f} val loss: {losses['val']:.4f}")

context = torch.zeros((1, 1), dtype=torch.long, device=device)
text = decode(model.generate(context, max_tokens=500)[0].tolist())
print(text)
with open('output.txt', 'w') as f:
    f.write(text)

10.807776 M parameters


  0%|          | 0/3000 [00:00<?, ?it/s]